To Do:
Clean up result and separate into result, score, and opponent score
Convert Game type to words

In [1]:
import os
import pandas as pd
import sqlalchemy
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

connection_string = os.getenv("LINODE_CONN_STRING")
connection_string = os.getenv("LINODE_CONN_STRING")
conn = sqlalchemy.create_engine(os.getenv("LINODE_CONN_STRING")).connect()

C:\Users\amarl\AppData\Local\Temp\ipykernel_23076\3569850963.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_sql("SELECT * FROM sports_data.game_stats", conn)

In [4]:
data['Win_Loss'] = data['result'].apply(lambda x: x[:1])

In [6]:
data['Win_Loss'].value_counts()

Win_Loss
L    52674
W    52674
Name: count, dtype: int64

In [14]:
def determine_scores(row):
    if row['Win_Loss'] == 'W':
        row['score'] = row['result'][3:].split('-')[0]
        row['opponent_score'] = row['result'][3:].split('-')[1]
    else:
        row['score'] = row['result'][3:].split('-')[1]
        row['opponent_score'] = row['result'][3:].split('-')[0]
    return row

In [15]:
data = data.apply(determine_scores, axis=1)

In [10]:
data['opponent_score'] = data['result'].apply(lambda x: x[3:].split('-')[1])

In [16]:
data[['result', 'Win_Loss', 'score', 'opponent_score']]

,result,Win_Loss,score,opponent_score
0,"L, 83-73",L,73,83
1,"W, 83-73",W,83,73
2,"L, 68-53",L,53,68
3,"W, 68-53",W,68,53
4,"W, 59-45",W,59,45
...,...,...,...,...
105343,"W, 76-67",W,76,67
105344,"W, 79-64",W,79,64
105345,"L, 79-64",L,64,79
105346,"L, 74-69",L,69,74


In [17]:
for i, row in data.iterrows():
    score = int(row['score'])
    opp_score = int(row['opponent_score'])
    if row['Win_Loss'] == 'W' and opp_score > score:
        print(f"Error at index {i}")
    elif row['Win_Loss'] == 'L' and score > opp_score:
        print(f"Error at index {i}")

In [19]:
# 0 is Non-Conference, 1 is conference, 2 is conference tourney, 3 is NCAA tourney (generally? might be exceptions where 3 is other post season tourney)
def convert_game_type(row):
    if row['game_type'] == 0:
        return 'Non-Conference'
    elif row['game_type'] == 1:
        return 'Conference'
    elif row['game_type'] == 2:
        return 'Conference Tourney'
    elif row['game_type'] == 3:
        return 'Post-Season Tourney'
    else:
        return 'Other'

In [20]:
data['game_type'] = data.apply(convert_game_type, axis=1)

In [21]:
data['game_type'].value_counts()

game_type
Conference             60412
Non-Conference         37870
Conference Tourney      4990
Post-Season Tourney     2076
Name: count, dtype: int64

In [26]:
data.shape[0]

105348

In [25]:
data.to_sql('game_stats_v2', sqlalchemy.create_engine(os.getenv("LINODE_CONN_STRING")), if_exists='replace', index=False, schema='sports_data')

348